In [1]:

import csv
import matplotlib as mlp
import torch.nn as nn
import numpy as np
import torch.optim as optim
import os
import random
import torch
import PIL
import pandas as pd
import cv2
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
import pathlib
import torch.nn.functional as F
import glob
import matplotlib.pyplot as plt
import shutil
import matplotlib.ticker as ticker
import heapq
from collections import defaultdict
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
X_train = []
Y_train = []
X_val = []
Y_val = []
X_test = []
Y_test = []

In [3]:
tsv_file = open("/kaggle/input/assignment3/aksharantar_sampled/tel/tel_train.csv")
read_tsv = csv.reader(tsv_file)

pad1 = "\t"
space1 = "\n"
for i in read_tsv:   
    Y_train.append(i[1])
    X_train.append(i[0])

X_train = np.array(X_train)

val_tsv_file = open("/kaggle/input/assignment3/aksharantar_sampled/tel/tel_valid.csv")
val_read_tsv = csv.reader(val_tsv_file)
pad2 = "\t"
space2 = "\n"
for i in val_read_tsv:
    Y_val.append(i[1])
    X_val.append(i[0])
X_val= np.array(X_val)

test_tsv_file = open("/kaggle/input/assignment3/aksharantar_sampled/tel/tel_test.csv")
test_read_tsv = csv.reader(test_tsv_file)
pad3 = "\t"
space3 = "\n"
for i in test_read_tsv:
    Y_test.append(i[1])
    X_test.append(i[0])

X_test = np.array(X_test)
trainsize=len(X_train)
testsize = len(X_test)
validationsize = len(X_val)

print("Number of Training samples:", trainsize)
print("Number of Test Samples:" , testsize)
print("Number of Validation Samples:", validationsize)

Number of Training samples: 51200
Number of Test Samples: 4096
Number of Validation Samples: 4096


In [4]:
Y_train = np.array(Y_train)
for i in range(Y_train.shape[0]):
    Y_train[i] = pad1 + Y_train[i] + space1

Y_val = np.array(Y_val)
for i in range(Y_val.shape[0]):
    Y_val[i] = pad2 + Y_val[i] + space2
    
Y_test = np.array(Y_test)
for i in range(Y_test.shape[0]):
    Y_test[i] = pad3+ Y_test[i] + space3


In [5]:
traintargetsize=len(Y_train)
testtargetsize = len(Y_test)
testtargetsize = len(Y_val)
print("TrainSize of Target ",traintargetsize)
print("TrainSize of Target ",testtargetsize)
print("TrainSize of Target ",testtargetsize)


TrainSize of Target  51200
TrainSize of Target  4096
TrainSize of Target  4096


In [6]:
input_corpus = set()
output_corpus = set()
input_corpus = set(char for word in X_train for char in word if char not in input_corpus)
output_corpus = set(char for word in Y_train for char in word if char not in output_corpus)
input_corpus.add(" ")
output_corpus.add(" ")
input_corpus = sorted(list(input_corpus))
output_corpus = sorted(list(output_corpus))
val_input_corpus = set()
val_output_corpus = set()
max_encoder_seq_length = max([len(txt) for txt in X_train]) + 2
max_decoder_seq_length = max([len(txt) for txt in Y_train])

print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
val_input_corpus = set(char for word in X_val for char in word if char not in val_input_corpus)
val_output_corpus = set(char for word in Y_val for char in word if char not in val_output_corpus)


Max sequence length for inputs: 30
Max sequence length for outputs: 21


In [7]:
num_encoder_tokens = len(input_corpus)
num_decoder_tokens = len(output_corpus)
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)

Number of unique input tokens: 27
Number of unique output tokens: 65


In [8]:


input_data = np.zeros((max_encoder_seq_length,len(X_train)), dtype="int64")
target_data = np.zeros((max_decoder_seq_length,len(X_train)), dtype="int64")
input_data_val = np.zeros((max_encoder_seq_length,len(X_val)), dtype="int64")
target_data_val = np.zeros((max_decoder_seq_length,len(X_val)), dtype="int64")

In [9]:
input_size_encoder = num_encoder_tokens
input_size_decoder = num_decoder_tokens
output_size = num_decoder_tokens


In [10]:
def create_char_index(corpus):
    char_index = defaultdict(int)
    for i, char in enumerate(corpus):
        if char not in char_index:
            char_index[char] = i
    return dict(char_index)

input_char_index = create_char_index(input_corpus)
output_char_index = create_char_index(output_corpus)

In [11]:
for i in range(len(X_train)):
    x = X_train[i]
    y = Y_train[i]
    count=0
    size = x.size
    for t, char in enumerate(x):
        input_data[t, i] = input_char_index[char]
    count=count+1 
    input_data[t + 1 :,i] = input_char_index[" "]
    count1=1
    for t, char in enumerate(y):
        target_data[t, i] = output_char_index[char]
        count1=count+1     
    target_data[t + 1 :,i] = output_char_index[" "]
    

data_type = torch.int64
data_type1=torch.float64
for i in range(len(X_val)):
    x=X_val[i]
    y=Y_val[i]
    count = 0
    size - x.size
    for t, char in enumerate(x):
        input_data_val[t, i] = input_char_index[char]
        count = count +1
    input_data_val[t + 1 :,i] = input_char_index[" "]
    
    count = 0
    for t, char in enumerate(y):
        target_data_val[t, i] = output_char_index[char]
        count = count +1
            
    target_data_val[t + 1 :,i] = output_char_index[" "]

input_data_val = torch.tensor(input_data_val,dtype=data_type)
target_data_val = torch.tensor(target_data_val,dtype=data_type)

In [12]:

input_data = torch.tensor(input_data,dtype=data_type)
reverse_input_char_index = {i: char for char, i in input_char_index.items()}
reverse_target_char_index = {i: char for char, i in output_char_index.items()}
target_data = torch.tensor(target_data,dtype=data_type)

In [13]:
#LSTM RUN Only
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout,opsize):
        super(Encoder, self).__init__()
        self.dropout,self.num_layers,self.bidirectional,self.embedding = nn.Dropout(dropout),num_layers,True,nn.Embedding(input_size, embedding_size)
        self.hidden_size,self.identity_init,self.init_h0 = hidden_size,torch.eye(hidden_size),nn.Linear(num_layers * 2, num_layers)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)
        self.fc_hidden,self.fc_cell = nn.Linear(hidden_size * 2, hidden_size), nn.Linear(hidden_size * 2, hidden_size)     

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size
        a = self.embedding(x)
        embedding = self.dropout(a)
        identity_init = self.identity_init
        embedded = self.embedding(x).view(1, 1, -1)
        # embedding shape: (seq_length, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)
        init_h0 = self.init_h0

        return hidden, cell
    def initHidden(self):
        value = 1
        return torch.zeros(value, value, self.hidden_size, device=device)
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(Decoder, self).__init__()
        self.dropout,self.num_layers,self.bidirectional,self.embedding = nn.Dropout(dropout), num_layers,True,nn.Embedding(input_size, embedding_size)
        self.hidden_size,self.identity_init,self.init_h0 = hidden_size,torch.eye(hidden_size), nn.Linear(num_layers * 2, num_layers)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)
        self.relu = nn.ReLU()

    def forward(self, x, hidden, cell):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x = x.unsqueeze(0)
        hid=hidden.squeeze(0)
        embedding = self.dropout(self.embedding(x))
        embed=embedding[0:]
        # embedding shape: (1, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs).squeeze(0)
        results=predictions[0:]
        results = self.softmax(predictions)
        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
       # predictions = predictions.squeeze(0)

        return predictions, hidden, cell
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.2):
        x = target[0]
        #batch_size = source.shape[1]
        #target_len = target.shape[0]
        variables = 0
        target_vocab_size = num_decoder_tokens

        outputs = torch.zeros(target.shape[0], source.shape[1], target_vocab_size).to(device)
        
        hidden, cell=self.encoder(source)
        ht = hidden[-1, :, :]
       # hidden, cell = self.encoder(source)

        # Grab the first input to the Decoder which will be <SOS> token
        res = cell[-1,:,:]

        for t in range(1, target.shape[0]):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell = self.decoder(x, hidden, cell)

            # Store next output prediction
            outputs[t] = output
            res = output[0:]
            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)
            #x = target[t] if random.random() < teacher_force_ratio else best_guess
            if random.random() < teacher_force_ratio:
                x = target[t]
            else:
                x = best_guess

        return outputs


In [14]:
#For GRU
class GRU_Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout,size):
        super(GRU_Encoder, self).__init__()
        self.dropout,self.num_layers,self.bidirectional,self.embedding = nn.Dropout(dropout),num_layers,True,nn.Embedding(input_size, embedding_size)
        self.hidden_size,self.identity_init,self.init_h0 = hidden_size,torch.eye(hidden_size),nn.Linear(num_layers * 2, num_layers)
        self.tanh = nn.LeakyReLU()
        self.fc_hidden,self.fc_cell = nn.Linear(hidden_size * 2, hidden_size), nn.Linear(hidden_size * 2, hidden_size) 
        self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout)
        self.tanh = nn.Tanh()

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size

        a = self.embedding(x)
        embedding = self.dropout(a)
        identity_init = self.identity_init
        embedded = self.embedding(x)
        #embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)
        outputs, hidden = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)
        # hidden shape: (num_layers, N, hidden_size)
        ht=hidden[0: :]
        return hidden

class GRU_Decoder(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(GRU_Decoder, self).__init__()
        self.dropout,self.num_layers,self.bidirectional,self.embedding = nn.Dropout(dropout), num_layers,True,nn.Embedding(input_size, embedding_size)
        self.hidden_size,self.identity_init,self.init_h0 = hidden_size,torch.eye(hidden_size), nn.Linear(num_layers * 2, num_layers)
        self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)
        self.relu = nn.ReLU()

       # self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):

        x = x.unsqueeze(0)
        a = self.embedding(x)
        embedding = self.dropout(a)
        # embedding shape: (1, N, embedding_size)

        outputs, hidden = self.rnn(embedding, hidden)
        
        predictions = self.fc(outputs)
        results = predictions
        results= self.softmax(results)
        predictions = predictions.squeeze(0)
        return predictions, hidden


class Seq2SeqGR(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2SeqGR, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.2):
        target_vocab_size = num_decoder_tokens
        count= 0 
        outputs = torch.zeros(target.shape[0], source.shape[1], target_vocab_size).to(device)
        source_vocab_size = num_encoder_tokens

        hidden= self.encoder(source)

        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]

        for t in range(1, target.shape[0]):
            # Use previous hidden, cell as context from encoder at start
            output, hidden = self.decoder(x, hidden)

            # Store next output prediction
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            res = output[0:]
            best_guess = output.argmax(1)
            #x = target[t] if random.random() < teacher_force_ratio else best_guess
            if random.random() < teacher_force_ratio:
                x = target[t]
            else:
                x = best_guess

        return outputs


In [15]:
## This cell is to Run RNN
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout):
        super(EncoderRNN, self).__init__()
        self.dropout,self.num_layers,self.bidirectional,self.embedding = nn.Dropout(dropout),num_layers,True,nn.Embedding(input_size, embedding_size)
        self.hidden_size,self.identity_init,self.init_h0 = hidden_size,torch.eye(hidden_size),nn.Linear(num_layers * 2, num_layers)
        self.tanh = nn.LeakyReLU()
        self.fc_hidden,self.fc_cell = nn.Linear(hidden_size * 2, hidden_size), nn.Linear(hidden_size * 2, hidden_size) 
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout)
        self.tanh = nn.Tanh()

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size
        a = self.embedding(x)
        embedding = self.dropout(a)
        identity_init = self.identity_init
        embedded = self.embedding(x)
        #embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)
        outputs, hidden = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)
        # hidden shape: (num_layers, N, hidden_size)
        return hidden
    




##For RNN
class DecoderRNN(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(DecoderRNN, self).__init__()
      #  self.softmax = nn.LogSoftmax(dim=1)
        self.dropout,self.num_layers,self.bidirectional,self.embedding = nn.Dropout(dropout), num_layers,True,nn.Embedding(input_size, embedding_size)
        self.hidden_size,self.identity_init,self.init_h0 = hidden_size,torch.eye(hidden_size), nn.Linear(num_layers * 2, num_layers)
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)
        self.relu = nn.ReLU()


    def forward(self, x, hidden):

        x = x.unsqueeze(0)
        a= self.embedding(x)

        embedding = self.dropout(a)

        outputs, hidden = self.rnn(embedding, hidden)
        
        predictions = self.fc(outputs)
        results = predictions
        results= self.softmax(results)
        predictions = predictions.squeeze(0)

        return predictions, hidden

In [16]:
import heapq
#     return word_t
def beam_findGR(model, word, norm,input_char_index, output_char_index, reverse_input_char_index,
                reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length,
                num_encoder_tokens, num_decoder_tokens, beam_width, device, length_penalty=0.6):

    word_t = ''

    word_val = 0
    data = np.zeros((max_encoder_seq_length, 1), dtype="int64")
    datatype = torch.int64
    for t in range(len(word)):
        char = word[t]
        data[t, word_val] = input_char_index[char]
    word_size =len(word)
    data[ word_size:, word_val] = input_char_index[" "]
    data = torch.tensor(data, dtype=datatype).to(device)

    with torch.no_grad():
        hidden = model.encoder(data)

    # Initialize beam
    
    value = 1
    ovalue=-1
    padding = '\t'
    initial_sequence = torch.tensor(np.array(output_char_index[padding]).reshape(value,)).to(device)
    zero = 0
    res =hidden.unsqueeze(0)
    beam = [(0.0, initial_sequence, hidden.unsqueeze(0))]  # [(score, sequence, hidden)]
    start ='\n'
    for _ in range(max_decoder_seq_length):
        candidates = []
        options=[]
        for score, seq, hidden in beam:
            count = 0
            last_token = seq[ovalue].item()
            if last_token == output_char_index[start]:
                # If the sequence ends with the end token, add it to the candidates
                candidates.append((score, seq, hidden))
                count =count+1
                continue

            x = torch.tensor(np.array(last_token).reshape(value,)).to(device)
            output, hidden = model.decoder(x, hidden.squeeze(0))
            res = output[0:]
            probabilities = F.softmax(output, dim=1)
            result =probabilities
            # Get the top-k probabilities and tokens
            topk_probs, topk_tokens = torch.topk(probabilities, k=beam_width)
            tops = topk_probs[0]
            topk = topk_tokens[0]
            for prob, token in zip(tops, topk):
                new_s= token.unsqueeze(0)
                new_seq = torch.cat((seq, new_s), dim=0)
                new_hidden = hidden.clone().unsqueeze(0)
                penalty = len(new_seq) - 1
                len_pen = length_penalty
                length_penalty_factor = ((penalty) / 5) **  len_pen   # Adjust penalty factor as needed
                candidates.append((score + torch.log(prob).item() / length_penalty_factor, new_seq, new_hidden))
                options.append((score + torch.log(prob).item() / length_penalty_factor))
        # Select top-k candidates based on the accumulated scores
        beam = heapq.nlargest(beam_width, candidates, key=lambda x: x[0])

    # Select the best sequence from the beam as the output
    best_res = ""
    best_score, best_sequence, _ = max(beam, key=lambda x: x[zero])
    word_t = ''.join([reverse_target_char_index[token.item()] for token in best_sequence[value:ovalue]])
    best_res = word_t

    return word_t

In [25]:
def beam_find(model, word,norm,input_char_index, output_char_index, reverse_input_char_index,
                reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length,
                num_encoder_tokens, num_decoder_tokens, beam_width, device, length_penalty=0.6):

    word_t = ''

    # Encode the input word
    word_val = 0
    data = np.zeros((max_encoder_seq_length, 1), dtype="int64")
    datatype = torch.int64
    for t in range(len(word)):
        char = word[t]
        data[t,  word_val] = input_char_index[char]
    word_size = len(word)
    data[word_size:, word_val] = input_char_index[" "]
    data = torch.tensor(data, dtype=torch.int64).to(device)

    with torch.no_grad():
        hidden,cell = model.encoder(data)

    # Initialize beam
   
    value = 1
    ovalue=-1
    padding = '\t'
    initial_sequence = torch.tensor(np.array(output_char_index[padding]).reshape(value,)).to(device)
    zero = 0
    res =hidden.unsqueeze(0)
    beam = [(0.0, initial_sequence, hidden.unsqueeze(0))]  # [(score, sequence, hidden)]
    start ='\n'
    for _ in range(max_decoder_seq_length):
        candidates = []
        for score, seq, hidden in beam:
            count = 0
            last_token = seq[ovalue].item()
            if last_token == output_char_index[start]:
                # If the sequence ends with the end token, add it to the candidates
                candidates.append((score, seq, hidden))
                count =count+1
                continue

            x = torch.tensor(np.array(last_token).reshape(value,)).to(device)
            output, hidden,cell = model.decoder(x, hidden.squeeze(0),cell)
            res = output[0:]
            probabilities = F.softmax(output, dim=1)
            result =probabilities
            # Get the top-k probabilities and tokens
            topk_probs, topk_tokens = torch.topk(probabilities, k=beam_width)
            tops = topk_probs[0]
            topk = topk_tokens[0]
            options=[]
            for prob, token in zip(tops, topk):
                new_s= token.unsqueeze(0)
                new_seq = torch.cat((seq, new_s), dim=0)
                new_hidden = hidden.clone().unsqueeze(0)
                penalty = len(new_seq) - 1
                len_pen = length_penalty
                length_penalty_factor = ((penalty) / 5) **  len_pen   # Adjust penalty factor as needed
                candidates.append((score + torch.log(prob).item() / length_penalty_factor, new_seq, new_hidden))
                options.append((score + torch.log(prob).item() / length_penalty_factor))
        # Select top-k candidates based on the accumulated scores
        beam = heapq.nlargest(beam_width, candidates, key=lambda x: x[0])

    # Select the best sequence from the beam as the output
    best_res = ""
    best_score, best_sequence, _ = max(beam, key=lambda x: x[zero])
    word_t = ''.join([reverse_target_char_index[token.item()] for token in best_sequence[value:ovalue]])
    best_res =word_t

    return word_t

In [26]:
def training(num_encoder_tokens,input_embedding_size, dp, cell_type, hidden_size, num_enc_layers, num_dec_layers,num_epochs,output_size,input_size_decoder,batch_size,beam_width):
    
    criterion = nn.CrossEntropyLoss()
    data_train = torch.split(input_data,batch_size,dim=1)
    target_train = torch.split(target_data,batch_size,dim=1)
    learning_rate=0.001
    if(cell_type=="LSTM"):
        encoder_net = Encoder(input_size_encoder,input_embedding_size, hidden_size, num_enc_layers,dp,output_size).to(device)
        decoder_net = Decoder(input_size_decoder,input_embedding_size,hidden_size,output_size,num_dec_layers,dp).to(device)

        model = Seq2Seq(encoder_net, decoder_net).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    elif(cell_type=="RNN" or cell_type =="GRU"):
        if(cell_type=="RNN"):
            encoder_net = EncoderRNN(input_size_encoder,input_embedding_size, hidden_size, num_enc_layers, dp).to(device)
            decoder_net = DecoderRNN(input_size_decoder,input_embedding_size,hidden_size,output_size,num_dec_layers,dp).to(device)
        else:
            encoder_net = GRU_Encoder(input_size_encoder,input_embedding_size, hidden_size, num_enc_layers, dp,output_size).to(device)
            decoder_net = GRU_Decoder(input_size_decoder,input_embedding_size,hidden_size,output_size,num_dec_layers,dp).to(device)
        total_words = len(X_val)
       
        model = Seq2SeqGR(encoder_net, decoder_net).to(device)    
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        
    for epoch in range(num_epochs):
        correct_output = 0
        print(f"[Epoch {epoch} / {num_epochs}]")

        model.eval()
        model.train()

        train_size = len(data_train)
        for i in range(train_size):
            neg = -1
            pos = 1
            a = data_train[i]
            b = target_train[i]
            # Get input and targets and get to cuda
            inp_data = a.to(device)
            target = b.to(device)

            # Forward prop
            output = model(inp_data, target)
            size = output.shape[2]
            temp = output[pos:]
            temp1 =temp.reshape(neg, size)

            output = temp1
            tar = target[pos:]
            target = tar.reshape(neg)

            optimizer.zero_grad()
            loss = criterion(output, target)

            # Back prop
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

                # Gradient descent step
            optimizer.step()

        model.eval()
        total_words = len(X_val)
        total_words_test = len(X_test)
        if(cell_type=="LSTM"):
            for i in range(total_words):
       
                decoded_sentence = beam_find(model,X_val[i], 10,input_char_index, output_char_index, reverse_input_char_index, 
                reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length, 
                num_encoder_tokens, num_decoder_tokens,beam_width,device)
                true_output = Y_val[i][1:-1]
                if true_output== decoded_sentence:
                     correct_output = correct_output + 1
       
            
        elif(cell_type=="GRU" or cell_type=="RNN"):
            for i in range(total_words):
                    decoded_sentence = beam_findGR(model,X_val[i], 10,input_char_index, output_char_index, reverse_input_char_index, 
                    reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length, 
                    num_encoder_tokens, num_decoder_tokens,beam_width,device)
                    true_output = Y_val[i][1:-1]
                    if true_output== decoded_sentence:
                        correct_output = correct_output + 1
        Calculated_test_accuracy = correct_output / total_words
        print(Calculated_test_accuracy)
        wandb.log({'val_accuracy' : Calculated_test_accuracy*100})

   

In [27]:
import wandb
wandb.login(key='17d991db26320e751b170877037d1067a164fe6d')
wandb.init(project="Assignment_3_F")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [28]:
def train():
    var1 = wandb.init()
    var2 = var1.config
    embedding_size_input = var2.embedding_size
    dropout_enc_dec = var2.dropout
    cell_typeval = var2.cell_type
    hidden_size_encdec = var2.hidden_size
    enc_num_layers = var2.enc_num_layers
    number_of_epochs = var2.epochs
    batch_size = var2.batch_size
    beam_width = var2.beam_search
    training(input_size_encoder ,embedding_size_input, dropout_enc_dec, cell_typeval,hidden_size_encdec, enc_num_layers,  enc_num_layers,number_of_epochs ,num_decoder_tokens,num_decoder_tokens,batch_size,beam_width)

In [29]:
training(input_size_encoder ,128, 0.2, "LSTM",128, 2, 2,10,num_decoder_tokens,num_decoder_tokens,32,1)

[Epoch 0 / 10]
0.00146484375
[Epoch 1 / 10]
0.08740234375
[Epoch 2 / 10]
0.180419921875
[Epoch 3 / 10]
0.23291015625
[Epoch 4 / 10]
0.288330078125
[Epoch 5 / 10]
0.326171875
[Epoch 6 / 10]
0.34912109375
[Epoch 7 / 10]
0.356201171875
[Epoch 8 / 10]
0.370361328125
[Epoch 9 / 10]
0.390380859375


In [30]:
sweep_config = {
    'method': 'bayes',
    'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
    'parameters': {'embedding_size': {'values': [128, 256, 512]},
                   'epochs' :{'values':[10,20,30,40]},
                   'hidden_size': {'values': [128, 256, 512]},
                   'batch_size': {'values': [128,256,512]},
                   'enc_num_layers': {'values': [1,2,3]},
                   'dropout': {'values': [0.1, 0.2, 0.3, 0.4]},
                   'dec_num_layers':{'values': [1,2,3]},
                   'cell_type': {'values': ['LSTM','GRU','RNN']},
                   'beam_search':{'values':[1,2,3,4,5]}
                }}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Assignment_3_Fin")
wandb.agent(sweep_id, train, count=50)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: bfuntv20
Sweep URL: https://wandb.ai/harshvrma/Assignment_3_Fin/sweeps/bfuntv20


wandb: Waiting for W&B process to finish... (success).
wandb: / 0.048 MB of 0.048 MB uploaded (0.000 MB deduped)
wandb: Run history:
wandb: val_accuracy ▁▃▄▅▆▇▇▇██
wandb: 
wandb: Run summary:
wandb: val_accuracy 39.03809
wandb: 
wandb: 🚀 View run pleasant-darkness-30 at: https://wandb.ai/harshvrma/Assignment_3_F/runs/8cxkag28
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230521_094255-8cxkag28/logs
wandb: Agent Starting Run: ypdg5ddw with config:
wandb: 	batch_size: 128
wandb: 	beam_search: 5
wandb: 	cell_type: GRU
wandb: 	dec_num_layers: 1
wandb: 	dropout: 0.4
wandb: 	embedding_size: 512
wandb: 	enc_num_layers: 3
wandb: 	epochs: 30
wandb: 	hidden_size: 512


[Epoch 0 / 30]
